<a href="https://colab.research.google.com/github/GonzaloMartin/UTN-Diplomatura-en-Python/blob/main/Python3%20-%20Nivel%20Intermedio/Python3%20Intermedio%20-%20Unidad%201.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DIPLOMATURA EN PYTHON

### UNIDAD DIDÁCTICA 1

- Módulo I – Nivel Intermedio.

#### _Resueltos de [@GonzaloMartin](https://github.com/GonzaloMartin)._

## Práctica:



### Ejercicio:

Tomando en cuenta la app del nivel inicial, separarla en módulos de vista y modelo.

In [ ]:
import re
import sqlite3
from tkinter import *
from tkinter import ttk

main = Tk()
main.title("Unidad 7- Tarea")

var_producto = StringVar()
var_descripcion = StringVar()

l_producto = Label(main, text="Producto")
l_producto.grid(row=0, column=0, sticky="W")
l_descripcion = Label(main, text="Descripción")
l_descripcion.grid(row=1, column=0, sticky="W")

input_producto = Entry(main, width=30, textvariable=var_producto)
input_producto.grid(row=0, column=2, sticky="E")
input_descripcion = Entry(main, width=30, textvariable=var_descripcion)
input_descripcion.grid(row=1, column=2, sticky="E")

def crear_base():
    con = sqlite3.connect('u7-tarea.db')
    return con

def crear_tabla(con):
    cursorObj = con.cursor()
    cursorObj.execute("CREATE TABLE IF NOT EXISTS productos (id integer PRIMARY KEY, producto text, descripcion text);")
    con.commit()

def guardar_en_base():
    con = crear_base()
    crear_tabla(con)
    cursorObj = con.cursor()

    producto = input_producto.get()
    descripcion = input_descripcion.get()

    if producto == "" and descripcion == "":
        print("Faltan datos por completar.")
        return

    if re.match(r"^[a-zA-Z0-9 ]+$", producto):
        print("Producto válido")

        data = (producto, descripcion)
        query = "INSERT INTO productos (producto, descripcion) VALUES(?, ?);"
        cursorObj.execute(query, data)
        con.commit()
        imprimir(data)
        actualizar_treeview(datos=data)
    else:
        print("Sólo caracteres alfanuméricos.")

def imprimir(datos):
    print(datos)

def actualizar_treeview(datos):
    tree.insert("", END, text="", values=(datos[0], datos[1]))


btn_agregar = Button(main, text="Aceptar", command=guardar_en_base)
btn_agregar.grid(row=3, column=2)

tree = ttk.Treeview(main)
tree["columns"] = ("Col1","Col2")

tree.column("#0", width=0, minwidth=0)
tree.column("Col1", width=100, minwidth=100)
tree.column("Col2", width=200, minwidth=200)
tree.heading("#0", text="", anchor="w")
tree.heading("Col1", text="Producto", anchor="w")
tree.heading("Col2", text="Descripción", anchor="w")

tree.grid(row=4, column=0, columnspan=3)

main.mainloop()

### Output:

Vista de consola, formulario y base de datos:

<img src="https://raw.githubusercontent.com/GonzaloMartin/UTN-Diplomatura-en-Python/main/assets/P1-U7-Ejercicio.PNG">